In [58]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import autograd, linalg
from torch.utils.data import DataLoader, TensorDataset
from sklearn.feature_extraction.text import CountVectorizer

In [59]:
# from google.colab import drive
# drive.mount('/content/drive')

In [60]:
if torch.cuda.is_available():
  print('Good to go!')
  device = torch.device('cuda:0')
else:
  print('Using CPU.')
  device = torch.device('cpu')

Good to go!


In [61]:
rcounter = CountVectorizer()
data_path = 'movies_extrainfos.dat'
removie_tid_map = {}
removie_tid_list = []
maxcontent = []
with open(data_path) as f:
  for i, line in enumerate(f):
    id = int(line.split('::')[0])
    info = line.split('::')[-2]
    words = info.split()
    info = [word for word in words if len(word)>8]
    info = ' '.join(info)
    maxcontent.append(info)
    removie_tid_map[id] = i
    removie_tid_list.append(id)
    #print(id, info)
# print(maxcontent)
maxdocs = np.array(maxcontent)
bag = rcounter.fit_transform(maxdocs)
# print(bag)
n_words = bag.shape[1]
print(rcounter.vocabulary_)
print(bag.shape)
# print(removie_tid_map)

{'especially': 3268, 'lightyear': 5492, 'jealousy': 5169, 'phillips': 6953, 'man': 5685, 'child': 1561, 'creatures': 2261, 'fishermen': 3628, 'something': 8645, 'african': 265, 'american': 389, 'friendship': 3804, 'strongest': 8958, 'different': 2663, 'directions': 2701, 'light': 5486, 'hearted': 4277, 'struggles': 8968, 'daughter': 2388, 'ascension': 603, 'perspective': 6918, 'comfortable': 1776, 'progression': 7320, 'rebirth': 7616, 'prey': 7240, 'neil': 6328, 'professional': 7300, 'robbery': 8038, 'homicide': 4425, 'division': 2871, 'retirement': 7963, 'including': 4723, 'mentor': 5917, 'never': 6345, 'corner': 2166, 'as': 601, 'suspenseful': 9135, 'ending': 3161, 'fairchild': 3487, 'chauffeur': 1529, 'invisible': 5098, 'branches': 1154, 'larrabee': 5394, 'matriarch': 5810, 'corporation': 2173, 'successful': 9027, 'communications': 1819, 'handsome': 4189, 'fun': 3839, 'loving': 5587, 'sabrina': 8118, 'magazine': 5644, 'blossomed': 1061, 'beautiful': 887, 'sophisticated': 8659, 'misc

In [62]:
f = open("content.txt", "w")
dicur_i_id = 0
brows, bcols = bag.nonzero()
for x, y, v in zip(brows, bcols, bag.data):
  if x != dicur_i_id:
    f.write('\n') 
    # f.write(maxcontent[live_u_id])
    # f.write('\n') 
    dicur_i_id += 1
  # print(f'{y}:{v} ')
  f.write(f'{y}:{v} ')
f.close()

In [63]:
data_path = 'user_list.txt'
moddata_list = []
with open(data_path) as f:
  for i, line in enumerate(f):
    moddata_list.append(int(line))
#print(moddata_list)

In [64]:
data_path = 'ratings.dat'
modelf_train = open("train_rating_util.txt", "w")
modelf_test = open("test_rating_util.txt", "w")
total_users = 6040
total_items = 3881
live_u_id = 1
is_train = 18
with open(data_path) as f:
  for i, line in enumerate(f):
    split = line.split('::')
    user_id = int(split[0])
    movie_id = int(split[1])
    rating = int(split[2])

    if movie_id not in removie_tid_list:
      continue

    if live_u_id != user_id:
      live_u_id += 1
      modelf_train.write('\n')
      modelf_test.write('\n')
      is_train = 18
    if is_train != 0:
      modelf_train.write(f'{removie_tid_map[movie_id]}:{user_id} ')
      is_train -= 1
    else:
      modelf_test.write(f'{removie_tid_map[movie_id]}:{rating} ')
modelf_train.close()
modelf_test.close()


In [65]:
print(len(moddata_list))
total_users = 1147
modelf_train = open("train_rating.txt", "w")
modelf_test = open("test_rating.txt", "w")

with open("train_rating_util.txt") as f:
  for i, line in enumerate(f):
    if (i+1) in moddata_list:
      modelf_train.write(line)

with open("test_rating_util.txt") as f:
  for i, line in enumerate(f):
    if (i+1) in user_list:
      modelf_test.write(line)

modelf_train.close()
modelf_test.close()

1147


In [66]:
def transform_file(dataset, shape):
    indices = [[], []]
    values = []

    with open(dataset) as f:
        for i, line in enumerate(f):
            id_and_ratings = line.split()

            for id_and_rating in id_and_ratings:
              id_and_rating_split = id_and_rating.split(':')
              indices[0].append(i)
              indices[1].append(int(id_and_rating_split[0]))
              values.append(float(id_and_rating_split[1]))

    x = torch.sparse_coo_tensor(indices, values, shape, dtype=torch.float32)
    return x

ratings_training_dataset = transform_file('train_rating.txt', (total_users, total_items))
# print(ratings_training_dataset)
ratings_test_dataset = transform_file('test_rating.txt', (total_users, total_items))
# print(ratings_test_dataset)

In [67]:
def bag_of_words(dataset, shape):
    indices = [[], []]
    values = []

    with open(dataset) as f:
        for doc_id, line in enumerate(f):
            tokens = line.split()

            for token in tokens:
                word_id, cnt = tuple(map(int, token.split(':')))
                indices[0].append(doc_id)
                indices[1].append(word_id)
                values.append(cnt)

    ret_val = torch.sparse_coo_tensor(indices, values, shape, dtype=torch.float32).to_dense()

    maxes, _ = ret_val.max(dim=1, keepdim=True)
    torch.clamp_min_(maxes, 1)
    ret_val /= maxes
    return ret_val

content_dataset = bag_of_words('content.txt', (total_items, n_words)).to(device)
if content_dataset.is_sparse:
    content_dataset = content_dataset.to_dense()
# print(content_dataset)

In [68]:
total_items, in_features = content_dataset.shape

In [69]:
class Autoencoder(nn.Module):
    def __init__(self, in_features, latent_size, dropout=0, activation=nn.Sigmoid()):
        super().__init__()

        encode = nn.Linear(in_features, latent_size)
        decode = nn.Linear(latent_size, in_features)

        decode.weight.data = encode.weight.data.t()

        self.dropout = nn.Dropout(dropout)
        self.activation = activation

        self.encode = nn.Sequential(
            encode,
            self.activation,
        )
        self.decode = nn.Sequential(
            self.dropout,
            decode,
        )

    def forward(self, x):
        latent = self.encode(x)
        reconstructed = self.decode(latent)
        return latent, reconstructed

class StackedAutoencoder(nn.Module):
    def __init__(self, autoencoders):
        super().__init__()

        self.autoencoders = nn.ModuleList(autoencoders)

    def forward(self, x):
        latent = self.encode(x)
        reconstructed = self.decode(latent)
        return latent, reconstructed

    def encode(self, x):
        for i, ae in enumerate(self.autoencoders):
            x = ae.encode(x)

            if i != len(self.autoencoders) - 1:
                x = ae.dropout(x)

        return x

    def decode(self, x):
        for i, ae in enumerate(reversed(self.autoencoders)):
            if i != 0:
                x = ae.activation(x)

            x = ae.decode(x)

        return x

In [70]:
class DataTransform:
    def __init__(self, dataset, transform):
        self._dataset = dataset
        self._transform = transform

    def __getitem__(self, i):
        return self._transform(self._dataset[i])

    def __len__(self):
        return len(self._dataset)

def bernoulli_corrupt(x, p):
    m = torch.rand_like(x) > p
    return x * m

def random_subset(x, k):
    id = torch.randperm(len(x))[:k]
    return torch.utils.data.Subset(x, id)

In [71]:
def train_stacked_autoencoder(stacked_autoencoder, dataset, corruption, epochs, batch_size, loss_fn, optimizer):
    cur_dataset = dataset

    for i, autoencoder in enumerate(stacked_autoencoder.autoencoders):
        print(f'Training autoencoder {i + 1}/{len(stacked_autoencoder.autoencoders)}')

        train_isolated_autoencoder(autoencoder, cur_dataset, corruption, epochs, batch_size, loss_fn, optimizer)

        with torch.no_grad():
            autoencoder.eval()
            cur_dataset = autoencoder.encode(cur_dataset[:])
            autoencoder.train()

    train_isolated_autoencoder(stacked_autoencoder, dataset, corruption, epochs, batch_size, loss_fn, optimizer)


def train_isolated_autoencoder(autoencoder, maxcontent, corruption, epochs, batch_size, loss_fn, optimizer):
    for epoch in range(epochs):
        print(f'epoch {epoch + 1}/{epochs}')

        dataset = DataTransform(
            maxcontent,
            lambda x: (bernoulli_corrupt(x, corruption), x),
        )
        train(lambda x: autoencoder(x)[1], dataset, loss_fn, batch_size, optimizer)

def train(model, dataset, loss_fn, batch_size, optimizer):
    dataloader = DataLoader(dataset, batch_size)
    size = len(dataset)

    for i, (xb, yb) in enumerate(dataloader):
        yb_pred = model(xb)
        loss = loss_fn(yb_pred, yb)

        if i % 100 == 0:
            current = i * batch_size
            print(f'  loss: {loss:>7f}  [{current:>5d}/{size:>5d}]')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [72]:
layer_sizes = [in_features] + [400] + [50]
print(f'{"x".join(map(str, layer_sizes))}')

autoencoders = [
    Autoencoder(in_features, out_features, dropout=0.3, activation=nn.Sigmoid()) for in_features, out_features in zip(layer_sizes, layer_sizes[1:])
]
sdae = StackedAutoencoder(autoencoders)
sdae.to(device)

10347x400x50


StackedAutoencoder(
  (autoencoders): ModuleList(
    (0): Autoencoder(
      (dropout): Dropout(p=0.3, inplace=False)
      (activation): Sigmoid()
      (encode): Sequential(
        (0): Linear(in_features=10347, out_features=400, bias=True)
        (1): Sigmoid()
      )
      (decode): Sequential(
        (0): Dropout(p=0.3, inplace=False)
        (1): Linear(in_features=400, out_features=10347, bias=True)
      )
    )
    (1): Autoencoder(
      (dropout): Dropout(p=0.3, inplace=False)
      (activation): Sigmoid()
      (encode): Sequential(
        (0): Linear(in_features=400, out_features=50, bias=True)
        (1): Sigmoid()
      )
      (decode): Sequential(
        (0): Dropout(p=0.3, inplace=False)
        (1): Linear(in_features=50, out_features=400, bias=True)
      )
    )
  )
)

In [73]:
optimizer = torch.optim.AdamW(sdae.parameters(), lr=1e-3, weight_decay=1e-4)

content_training_dataset = random_subset(content_dataset, int(total_items * 0.8))

print(f'Pretraining SDAE with mse loss')
train_stacked_autoencoder(sdae, content_training_dataset, corruption=0.3, epochs=20, batch_size=128, loss_fn=nn.MSELoss(), optimizer=optimizer)

Pretraining SDAE with mse loss
Training autoencoder 1/2
epoch 1/20
  loss: 0.005876  [    0/ 3104]
epoch 2/20
  loss: 0.002399  [    0/ 3104]
epoch 3/20
  loss: 0.001487  [    0/ 3104]
epoch 4/20
  loss: 0.001222  [    0/ 3104]
epoch 5/20
  loss: 0.001026  [    0/ 3104]
epoch 6/20
  loss: 0.000880  [    0/ 3104]
epoch 7/20
  loss: 0.000765  [    0/ 3104]
epoch 8/20
  loss: 0.000685  [    0/ 3104]
epoch 9/20
  loss: 0.000630  [    0/ 3104]
epoch 10/20
  loss: 0.000594  [    0/ 3104]
epoch 11/20
  loss: 0.000571  [    0/ 3104]
epoch 12/20
  loss: 0.000557  [    0/ 3104]
epoch 13/20
  loss: 0.000550  [    0/ 3104]
epoch 14/20
  loss: 0.000545  [    0/ 3104]
epoch 15/20
  loss: 0.000543  [    0/ 3104]
epoch 16/20
  loss: 0.000545  [    0/ 3104]
epoch 17/20
  loss: 0.000549  [    0/ 3104]
epoch 18/20
  loss: 0.000549  [    0/ 3104]
epoch 19/20
  loss: 0.000549  [    0/ 3104]
epoch 20/20
  loss: 0.000549  [    0/ 3104]
Training autoencoder 2/2
epoch 1/20
  loss: 0.227212  [    0/ 3104]
epoch

In [74]:
def train_cdl_autoencoder(autoencoder, maxcontent, latent_items, corruption, batch_size, recon_loss_fn, latent_loss_fn, optimizer):
    dataset = DataTransform(
        TensorDataset(latent_items, maxcontent),
        lambda x: (bernoulli_corrupt(x[1], corruption), x),
    )

    def loss_fn(pred, target):
        latent_pred, recon_pred = pred
        latent_target, recon_target = target
        return recon_loss_fn(recon_pred, recon_target) + latent_loss_fn(latent_pred, latent_target)

    train(autoencoder, dataset, loss_fn, batch_size, optimizer)

In [75]:
class MatrixFactorization:
    def __init__(self, target_shape, latent_size):
        super().__init__()

        self.U = torch.empty((target_shape[0], latent_size))
        self.V = torch.empty((target_shape[1], latent_size))
        self.latent_size = latent_size

        nn.init.normal_(self.U, 0, 0.1)
        nn.init.normal_(self.V, 0, 0.1)

    def estimate(self):
        return self.U @ self.V.t()

In [76]:
class MatrixFactorizationOptimizer:
    def __init__(self, model, ratings, c_a, c_b, l_u, l_v):
        self.model = model
        self.ratings = ratings

        self.c_a = c_a
        self.c_b = c_b

        self.l_u = l_u
        self.l_v = l_v

        self._ratings_nonzero_rows = [row.coalesce().indices().squeeze(0) for row in ratings]
        self._ratings_nonzero_cols = [col.coalesce().indices().squeeze(0) for col in ratings.t()]

    def step(self, latent_item_target):
        self.update_users()
        self.update_items(latent_item_target)

    def update_users(self):
        U, V, c_a, c_b = self.model.U, self.model.V, self.c_a, self.c_b

        A_base = c_b * V.t() @ V + self.l_u * torch.eye(self.model.latent_size)
        for j in range(len(U)):
            rated_idx = self._ratings_nonzero_rows[j]
            W = V[rated_idx, :]
            A = (c_a - c_b) * W.t() @ W + A_base
            b = c_a * W.t().sum(dim=1)

            linalg.solve(A, b, out=U[j].unsqueeze(1))

    def update_items(self, latent_items_target):
        U, V, c_a, c_b = self.model.U, self.model.V, self.c_a, self.c_b
        latent_items_target = latent_items_target * self.l_v

        A_base = c_b * U.t() @ U + self.l_v * torch.eye(self.model.latent_size)
        for j in range(len(V)):
            rated_idx = self._ratings_nonzero_cols[j]
            if len(rated_idx) == 0:
                A = A_base
                b = latent_items_target[j]
            else:
                W = U[rated_idx, :]
                A = (c_a - c_b) * W.t() @ W + A_base
                b = c_a * W.t().sum(dim=1) + latent_items_target[j]

            linalg.solve(A, b, out=V[j].unsqueeze(1))

    def loss(self, latent_items_target):
        conf_mat = self.c_b * torch.ones(self.ratings.shape) + (self.c_a - self.c_b) * self.ratings

        loss_v = self.l_v * F.mse_loss(self.model.V, latent_items_target, reduction='sum') / 2
        loss_u = self.l_u * self.model.U.square().sum() / 2
        loss_r = (conf_mat * (self.model.estimate() - self.ratings).square()).sum() / 2

        loss = loss_v + loss_u + loss_r
        return loss

def AutoencoderLoss(l_v, l_n):
    return lambda pred, target: l_v / l_n * F.mse_loss(pred, target)

In [77]:
def train_model(sdae, mfm, content, ratings, optimizer, recon_loss_fn, epochs, batch_size, device=None, max_iters=10):
    a=1.2
    b=0.05
    l_u=15
    l_v=25.0
    l_n=1e4
    l_w=1e-4
    mfm_optim = MatrixFactorizationOptimizer(mfm, ratings, a, b, l_u, l_v)
    latent_loss_fn = AutoencoderLoss(l_v, l_n)

    for epoch in range(epochs):
        print(f'Starting epoch {epoch + 1}/{epochs}')

        with autograd.no_grad():
            sdae.eval()
            latent_items_target, recon = sdae(content)
            latent_items_target = latent_items_target.cpu()
            sdae.train()

        mfm_optim.step(latent_items_target)

        if epoch % 3 == 0:
            loss = mfm_optim.loss(latent_items_target).item() 
            loss += l_n / 2 * F.mse_loss(recon, content, reduction='sum').item()
            loss += l_w / 2 * sum(p.square().sum() for p in sdae.parameters()).item()
            print(f'  neg_likelihood: {loss}')

        train_cdl_autoencoder(sdae, content, mfm.V.to(device), 0.3, batch_size, recon_loss_fn, latent_loss_fn, optimizer)

    with autograd.no_grad():
        sdae.eval()
        latent_items_target = sdae.encode(content).cpu()

    prev_loss = None
    for i in range(max_iters):
        mfm_optim.step(latent_items_target)
        loss = mfm_optim.loss(latent_items_target)
        if prev_loss is not None and (prev_loss - loss) / loss < 1e-4:
            break

        prev_loss = loss

In [78]:
mfm = MatrixFactorization(target_shape=ratings_training_dataset.shape, latent_size=50)
print(content_dataset.shape, ratings_training_dataset.shape)

print('Training with recon loss mse')
train_model(sdae, mfm, content_dataset, ratings_training_dataset, optimizer=optimizer, recon_loss_fn=nn.MSELoss(), epochs=10, batch_size=128, device=device, max_iters=10)

torch.Size([3881, 10347]) torch.Size([1147, 3881])
Training with recon loss mse
Starting epoch 1/10


<ipython-input-76-d45a49ea68f1>:29: UserWarning: An output with one or more elements was resized since it had shape [50, 1], which does not match the required output shape [50]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:26.)
  linalg.solve(A, b, out=U[j].unsqueeze(1))
<ipython-input-76-d45a49ea68f1>:46: UserWarning: An output with one or more elements was resized since it had shape [50, 1], which does not match the required output shape [50]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:26.)
  linalg.solve(A, b, out

  neg_likelihood: 787771288205651.2
  loss: 0.000689  [    0/ 3881]
Starting epoch 2/10
  loss: 0.000762  [    0/ 3881]
Starting epoch 3/10
  loss: 0.000743  [    0/ 3881]
Starting epoch 4/10
  neg_likelihood: 787669623667062.4
  loss: 0.000715  [    0/ 3881]
Starting epoch 5/10
  loss: 0.000712  [    0/ 3881]
Starting epoch 6/10
  loss: 0.000707  [    0/ 3881]
Starting epoch 7/10
  neg_likelihood: 787669756325923.1
  loss: 0.000708  [    0/ 3881]
Starting epoch 8/10
  loss: 0.000703  [    0/ 3881]
Starting epoch 9/10
  loss: 0.000705  [    0/ 3881]
Starting epoch 10/10
  neg_likelihood: 787669889443660.9
  loss: 0.000703  [    0/ 3881]


In [79]:
k=300
test = ratings_test_dataset.to_dense()
thres = 4
test[test <= thres] = 0
test[test > thres] = 1
values, indices = torch.topk(mfm.estimate(), k)
gathered = test.gather(1, indices)
recall = gathered.sum(dim=1) / test.sum(dim=1)
denom = test.sum(dim=1)
count = 0
sum_val = 0
for i in range(recall.shape[0]):
  if denom[i] != 0:
    count += 1
    sum_val += recall[i]
print("Avg Recall: ", sum_val/count)

Avg Recall:  tensor(0.3345)
